## 핵심 아이디어
1. 메뉴명을 명사 단위로 토큰화
2. 토큰화한 메뉴명을 빈도분석
3. 빈도 내림차순으로 나열한 뒤 특정 비율의 토큰을 채택
4. 채택한 토큰으로 끝나는 메뉴명을 토큰 카테고리로 분류

## 간단하지만 강력한 알고리즘
대부분 한국어 메뉴명이 분류로 종결된다는 특성을 이용하였습니다  
ex)오곡라떼 => '라떼'라는 분류명으로 종결됨

따라서 이렇게 1차 분류를 시도해보고, 오분류되었거나 미분류된 음식명을 2차 분류해보는건 어떠할지?  

In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 35.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# 데이터 임포팅 
df = pd.read_csv('menu_concated_v1.3')
df.dropna(inplace=True)
menu_list = df['menu'].tolist()

In [4]:
# 명사 단위 토큰화
from konlpy.tag import Okt
from tqdm import tqdm

t = Okt()
menu_list = [t.nouns(word) for word in tqdm(menu_list)]

100%|██████████| 192332/192332 [02:09<00:00, 1489.51it/s]


In [5]:
# 토큰화된 리스트의 리스트 내 리스트 제거 
menu_list = [v for word in menu_list for v in word]
menu_list[:10]

['정식', '짜장면', '짬뽕', '우동', '울면', '간짜장', '사천짜장', '삼선짜장', '유니짜장', '삼선']

In [6]:
# 토큰 빈도분석 
count_dict = {}
for i in tqdm(menu_list):
  if i not in count_dict:
    count_dict[i] = 1
  else:
    count_dict[i] += 1

100%|██████████| 365109/365109 [00:00<00:00, 2004534.47it/s]


In [7]:
# 토큰을 빈도 내림차순으로 정렬
items = count_dict.items()
sorted_items = sorted(items, key=lambda x: x[1], reverse=True) 

# 빈도 상위 10퍼센트의 토큰만 선택
cut_line = int(len(sorted_items) / 10)

# 단, 두글자 이상인 토큰만 선택 (1글자 토큰은 결과를 곱창내놓았음;; 하지만 불용어 설정을 잘 한다면??)
outliner_vocab = [k for k, v in sorted_items[:cut_line] if len(k)!=1]
outliner_vocab[:10]

['라떼', '치즈', '세트', '치킨', '추가', '맥주', '소주', '튀김', '새우', '볶음']

In [8]:
# 전체 메뉴명을 중복 없이 리스트로 가져옴 
menu = df['menu'].tolist()
menu = list(set(menu))
len(menu)

51437

In [9]:
#토큰명으로 끝나는 메뉴를 토큰 카테고리로 분류!!
menu_cluster = {}

for food_token in outliner_vocab:
  for food_name in menu:
    if food_name not in menu_cluster: # 이미 분류된 메뉴는 건들지 않음

      # 메뉴명이 토큰명보다 길면서 동시에 토큰명으로 끝나는 메뉴를 토큰 카테고리로 분류 
      if len(food_token) <= len(food_name) and food_name[-len(food_token):] == food_token:
        menu_cluster[food_name] = food_token

In [10]:
# 분류된 메뉴를 DataFrame화 
df_menu = pd.DataFrame(columns=['title', 'category'], data=menu_cluster.items())
df_menu['category'].unique()

array(['라떼', '치즈', '세트', '치킨', '추가', '맥주', '소주', '튀김', '새우', '볶음', '이드',
       '아메리카노', '피자', '구이', '스무디', '볶음밥', '불고기', '초코', '사리', '요거트', '음료수',
       '크림', '바닐라', '자몽', '레몬', '밀크', '해물', '우동', '공기밥', '딸기', '전골',
       '돈까스', '카라멜', '과일', '짬뽕', '라면', '정식', '샐러드', '만두', '한우', '김밥',
       '탕수육', '양념', '고구마', '오징어', '족발', '김치', '삼겹살', '감자', '갈비', '카페모카',
       '메뉴', '된장찌개', '떡볶이', '녹차', '막걸리', '카푸치노', '주스', '소고기', '망고',
       '김치찌개', '돼지', '보쌈', '수육', '에스프레소', '스테이크', '파스타', '커피', '브루',
       '아이스티', '비빔밥', '낙지', '칼국수', '전복', '블랙', '콜라', '냉면', '마끼아또', '순대',
       '음료', '블루베리', '야채', '두부', '허니', '마늘', '오리', '참치', '간장', '민트', '등심',
       '오뎅', '허브', '복분자', '고기', '제육', '베이컨', '얼그레이', '닭발', '스페셜', '버섯',
       '매운탕', '고추', '초밥', '우럭', '코스', '광어', '사이다', '그린티', '바나나', '쥬스',
       '찌개', '오렌지', '연어', '조림', '국밥', '국수', '철판', '모카', '동태', '백세주',
       '샌드위치', '갈릭', '화이트', '토마토', '유자', '아이스', '삼선', '계란', '매콤', '마리',
       '홍차', '잡채', '삼겹', '어묵', '물회', '해장국', '꼬치', '헤이즐넛', '생선', '버거',
 

In [11]:
pd.set_option('display.max_row', 100)

In [12]:
df_menu.sample(100)

,title,category
34009,채소,채소
26652,등심돈카츠안심,안심
33338,밀키베리무스,무스
3873,쌈야채고추마늘추가,추가
20385,불고기치즈그릴드샌드위치,샌드위치
10329,닭전골,전골
28744,베트남라이스,라이스
11739,청담순대국정식,정식
12380,치킨시저샐러드,샐러드
24373,오룡해삼,해삼


# 핵심 아이디어
1. 이전 토큰 종결 분류에서 계속된다  

2. 특정 토큰과 가장 가까운 토큰을 몇 개 추린다  
ex) 해장국 => 뼈, 콩나물  

3. 특정 토큰으로 분류된 음식명에 가까운 토큰이 포함되어 있나 조사한다  
ex) 뼈다귀 해장국 => 해장국으로 분류됨 + 뼈 토큰 포함  

4. 그럼 그 음식명에 속성을 추가해준다  
ex) 뼈다귀 해장국 => 해장국 속성 + 뼈 속성  

5. 두 속성이 동시에 포함된 음식명을 전체 음식 목록에서 찾는다  
ex) 해장국과 뼈가 동시에 포함된 음식명 = 뼈다귀 해장국  

6. 세부 분류를 한다  
ex) 뼈해장국을 해장국 > 뼈해장국으로 분류 


In [16]:
# 국밥을 예시로 시도해본다
df_menu[df_menu['category'] == '국밥']

,title,category
19520,특미마라국밥,국밥
19521,시래기순대국밥,국밥
19522,매생이국밥,국밥
19523,아우내국밥,국밥
19524,얼큰차돌국밥,국밥
...,...,...
19709,선지국밥,국밥
19710,수구레국밥,국밥
19711,송이국밥,국밥
19712,얼큰순대국밥,국밥


In [18]:
# 국밥 리스트 생성 
gookbab = df_menu[df_menu['category']=='국밥']['title'].tolist()
gookbab[:10]

['특미마라국밥',
 '시래기순대국밥',
 '매생이국밥',
 '아우내국밥',
 '얼큰차돌국밥',
 '순두부국밥',
 '장터따로국밥',
 '하노이국밥',
 '내장섞어국밥',
 '한우우거지국밥']

In [21]:
# 국밥 명사단위로 토큰화 후 리스트 내 리스트 제거 
gookbab_tokenized = [t.nouns(word) for word in gookbab]
gookbab_tokenized = [w for word in gookbab_tokenized for w in word]
gookbab_tokenized[:10]

['특', '국밥', '시래기', '순대', '국밥', '생', '국밥', '아우', '국밥', '얼']

In [22]:
# 토큰화된 국밥 리스트 분석
import nltk
gookbab_ko = nltk.Text(gookbab_tokenized, name='국밥 토큰')

In [35]:
# 국밥 토큰 개수와 빈도 분석 
gookbab_similar = gookbab_ko.vocab().most_common(15)
len(gookbab_ko), gookbab_similar

(446,
 [('국밥', 164),
  ('순대', 19),
  ('콩나물', 18),
  ('얼', 15),
  ('한우', 9),
  ('소고기', 9),
  ('장터', 8),
  ('굴국밥', 8),
  ('김치', 7),
  ('장국밥', 7),
  ('돼지국밥', 7),
  ('시래기', 5),
  ('우거지', 5),
  ('고기', 5),
  ('소머리국밥', 5)])

In [37]:
# 국밥 종류 생성
kind_of_gookbab = [value for value, num in gookbab_similar if len(value) != 1]
kind_of_gookbab

['국밥',
 '순대',
 '콩나물',
 '한우',
 '소고기',
 '장터',
 '굴국밥',
 '김치',
 '장국밥',
 '돼지국밥',
 '시래기',
 '우거지',
 '고기',
 '소머리국밥']

In [40]:
for token in kind_of_gookbab[1:]: #kind_of_gookbab[0] = '국밥'
  for menu in gookbab:
    if token in menu:
      print(f'{menu}: {token}')

시래기순대국밥: 순대
부산순대국밥: 순대
순대고기국밥: 순대
순대국밥: 순대
매운고추순대국밥: 순대
순대만국밥: 순대
김치순대국밥: 순대
따로순대국밥: 순대
순대뽈살국밥: 순대
올순대국밥: 순대
토종순대국밥: 순대
우거지순대국밥: 순대
황태순대국밥: 순대
콩나물순대국밥: 순대
특매운순대국밥: 순대
찰순대국밥: 순대
순대지옥국밥: 순대
얼큰장순대국밥: 순대
사골순대국밥: 순대
고기순대섞어국밥: 순대
흰순대국밥: 순대
한우사골순대국밥: 순대
얼큰이순대국밥: 순대
매운순대국밥: 순대
특순대국밥: 순대
순대국국밥: 순대
장순대국밥: 순대
살고기순대국밥: 순대
순대섞어국밥: 순대
얼큰순대국밥: 순대
양지콩나물국밥: 콩나물
김치콩나물따로국밥: 콩나물
콩나물김치국밥: 콩나물
한우콩나물국밥: 콩나물
불고기콩나물국밥: 콩나물
황태콩나물국밥: 콩나물
전주콩나물국밥: 콩나물
콩나물따로국밥: 콩나물
콩나물순대국밥: 콩나물
어린이콩나물국밥: 콩나물
전주식콩나물국밥: 콩나물
김치콩나물국밥: 콩나물
굴콩나물국밥: 콩나물
콩나물국밥: 콩나물
따로굴콩나물국밥: 콩나물
전주끊이는식콩나물국밥: 콩나물
전주남부시장식콩나물국밥: 콩나물
콩나물황태해장국밥: 콩나물
한우우거지국밥: 한우
한우콩나물국밥: 한우
한우장국밥: 한우
한우소머리국밥: 한우
한우소국밥: 한우
한우사골순대국밥: 한우
한우장터국밥: 한우
한우얼큰소머리국밥: 한우
한우따로국밥: 한우
한우시골장터국밥: 한우
한우국밥: 한우
구월산소고기국밥: 소고기
얼큰소고기국밥: 소고기
소고기국밥: 소고기
연변소고기국밥: 소고기
소고기따로국밥: 소고기
장터소고기국밥: 소고기
양평소고기국밥: 소고기
한끼소고기국밥: 소고기
얼큰소고기가마솥국밥: 소고기
장터따로국밥: 장터
소뼈장터국밥: 장터
한우장터국밥: 장터
장터소고기국밥: 장터
장터국밥: 장터
한우시골장터국밥: 장터
시골장터국밥: 장터
장터소국밥: 장터
들깨굴국밥: 굴국밥
하얀굴국밥: 굴국밥
따로매생이굴국밥: 굴국밥
굴국밥: 굴국밥
매생이굴국밥: 굴국밥
통영굴국밥: 굴국밥
얼큰굴국밥: 굴국밥
따로굴국밥